In [19]:
import requests
import pandas as pd
import requests
import os
os.chdir("/Users/apple/Desktop/DEV/portfolio/crypto-app")


In [37]:
from dotenv import load_dotenv
import os

load_dotenv()


True

In [38]:
coins=["bitcoin","ethereum","solana"]

url="https://api.coingecko.com/api/v3/coins/markets"
params={
    "vs_currency": "usd",
    "ids":",".join(coins),
    "price_change_percentage":"1h,24h,7d,30d"
}
headers={
    "accept": "application/json",
    "x-cg-demo-api-key": os.getenv("COINGECKO_API_KEY")
}

In [39]:
import requests
import pandas as pd
import json
from datetime import datetime
import os

def get_crypto_data(vs_currency="usd", coin_list=["bitcoin", "ethereum", "solana"], save_raw=True):
    url = "https://api.coingecko.com/api/v3/coins/markets"
    
    params = {
        "vs_currency": vs_currency,
        "ids": ",".join(coin_list)
    }
    
    response = requests.get(url, params=params)
    response.raise_for_status()
    data = response.json()
    
    # --- Bronze layer: save raw JSON in ./data/bronze/ ---
    if save_raw:
        base_dir = os.path.join(os.getcwd(), "data", "bronze", "coingecko")
        os.makedirs(base_dir, exist_ok=True)
        
        timestamp = datetime.utcnow().strftime("%Y-%m-%dT%H-%M-%S")
        file_path = os.path.join(base_dir, f"markets_{timestamp}.json")
        
        with open(file_path, "w") as f:
            json.dump(data, f, indent=2)
        
        print(f"✅ Raw JSON saved to {file_path}")
    
    # --- Silver layer: return as DataFrame ---
    df = pd.DataFrame(data)
    return df


In [40]:
coingecko_raw_df = get_crypto_data()
coingecko_raw_df


✅ Raw JSON saved to /Users/apple/Desktop/DEV/PORTFOLIO/crypto-app/data/bronze/coingecko/markets_2025-09-03T21-41-25.json


,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
0,bitcoin,btc,Bitcoin,https://coin-images.coingecko.com/coins/images...,111907.00,2228854468531,1,2228854468531,36789338128,112502.00,...,1.991536e+07,21000000.0,124128.00,-9.83797,2025-08-14T00:37:02.582Z,67.810000,1.649465e+05,2013-07-06T00:00:00.000Z,None,2025-09-03T21:41:01.440Z
1,ethereum,eth,Ethereum,https://coin-images.coingecko.com/coins/images...,4460.87,538804792549,2,538804792549,29881352402,4486.12,...,1.207054e+08,NaN,4946.05,-9.79726,2025-08-24T19:21:03.333Z,0.432979,1.030313e+06,2015-10-20T00:00:00.000Z,"{'times': 52.295512345227664, 'currency': 'btc...",2025-09-03T21:41:02.600Z
2,solana,sol,Solana,https://coin-images.coingecko.com/coins/images...,208.73,113014896030,6,127190541730,7435949609,212.41,...,6.089000e+08,NaN,293.31,-28.67399,2025-01-19T11:15:27.957Z,0.500801,4.167460e+04,2020-05-11T19:35:23.449Z,None,2025-09-03T21:41:05.309Z


In [31]:
coingecko_raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 26 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                3 non-null      object 
 1   symbol                            3 non-null      object 
 2   name                              3 non-null      object 
 3   image                             3 non-null      object 
 4   current_price                     3 non-null      float64
 5   market_cap                        3 non-null      int64  
 6   market_cap_rank                   3 non-null      int64  
 7   fully_diluted_valuation           3 non-null      int64  
 8   total_volume                      3 non-null      int64  
 9   high_24h                          3 non-null      float64
 10  low_24h                           3 non-null      float64
 11  price_change_24h                  3 non-null      float64
 12  price_change

In [27]:
def enforce_schema(df):
    schema = {
        "id": "string",
        "symbol": "string",
        "name": "string",
        "image": "string",
        "current_price": "float64",
        "market_cap": "Int64",   # nullable int
        "market_cap_rank": "Int64",
        "fully_diluted_valuation": "Int64",
        "total_volume": "Int64",
        "high_24h": "float64",
        "low_24h": "float64",
        "price_change_24h": "float64",
        "price_change_percentage_24h": "float64",
        "market_cap_change_24h": "Int64",
        "market_cap_change_percentage_24h": "float64",
        "circulating_supply": "float64",
        "total_supply": "float64",
        "max_supply": "float64",
        "ath": "float64",
        "ath_change_percentage": "float64",
        "last_updated": "datetime64[ns]"
    }

    # apply schema selectively (ignore cols not in dict)
    for col, dtype in schema.items():
        if col in df.columns:
            if dtype.startswith("datetime"):
                df[col] = pd.to_datetime(df[col], errors="coerce")
            else:
                df[col] = df[col].astype(dtype, errors="ignore")
    return df


In [32]:
silver_coingecko_df = enforce_schema(coingecko_raw_df)
print(silver_coingecko_df.dtypes)


id                                       string[python]
symbol                                   string[python]
name                                     string[python]
image                                    string[python]
current_price                                   float64
market_cap                                        Int64
market_cap_rank                                   Int64
fully_diluted_valuation                           Int64
total_volume                                      Int64
high_24h                                        float64
low_24h                                         float64
price_change_24h                                float64
price_change_percentage_24h                     float64
market_cap_change_24h                             Int64
market_cap_change_percentage_24h                float64
circulating_supply                              float64
total_supply                                    float64
max_supply                                      

In [33]:
silver_coingecko_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 26 columns):
 #   Column                            Non-Null Count  Dtype              
---  ------                            --------------  -----              
 0   id                                3 non-null      string             
 1   symbol                            3 non-null      string             
 2   name                              3 non-null      string             
 3   image                             3 non-null      string             
 4   current_price                     3 non-null      float64            
 5   market_cap                        3 non-null      Int64              
 6   market_cap_rank                   3 non-null      Int64              
 7   fully_diluted_valuation           3 non-null      Int64              
 8   total_volume                      3 non-null      Int64              
 9   high_24h                          3 non-null      float64            

In [ ]:
DB_USERNAME = os.getenv('DB_USERNAME', 'postgres')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_HOST = os.getenv('DB_HOST', 'localhost')
DB_PORT = os.getenv('DB_PORT', '5432')
DB_NAME = os.getenv('DB_NAME', 'ben')

In [0]:
%sql
-- DROP TABLE IF EXISTS raw.coingecko_markets_bronze

In [0]:
%sql
SELECT * FROM bronze_coingecko

In [0]:
%sql
CREATE OR REPLACE VIEW silver_coingecko AS
SELECT cg.id,cg.symbol,cg.name,cg.image,
        CAST(cg.current_price AS DOUBLE) current_price,
        CAST(cg.market_cap AS DOUBLE) market_cap,
        CAST(cg.market_cap_rank AS INT) market_cap_rank,
        CAST(cg.price_change_percentage_24h AS DOUBLE) price_change_percentage_24h,
        CAST(cg.market_cap_change_percentage_24h AS DOUBLE) market_cap_change_percentage_24h,
        CAST(cg.circulating_supply AS DOUBLE) circulating_supply,
        CAST(cg.ath AS DOUBLE) ath,
        CAST(cg.ath_change_percentage AS DOUBLE) ath_change_percentage,
        CAST(cg.ath_date AS TIMESTAMP) ath_date,
        CAST(cg.atl AS DOUBLE) atl,
        CAST(cg.atl_change_percentage AS DOUBLE) atl_change_percentage,
        CAST(cg.atl_date AS TIMESTAMP) atl_date
FROM bronze_coingecko cg

In [0]:
%sql
SELECT * FROM silver_coingecko

In [0]:
%sql
select * from silver_coindesk_data

In [0]:
%sql
WITH cte AS (
  SELECT
    cg.symbol,
    MAX(cd.current_price) AS current_price,
    MAX(ROUND(cd.day_high, 2)) AS day_high,
    MAX(ROUND(cd.day_low, 2))  AS day_low
  FROM raw.silver_coingecko_data cg
  LEFT JOIN silver_coindesk_data cd
    ON cg.symbol = cd.symbol
  GROUP BY cg.symbol
),

volatility_calc AS (
  SELECT
    symbol,
    ROUND(STDDEV(current_price), 2) AS volatility
  FROM silver_coindesk_data
  GROUP BY symbol
)

SELECT
  cte.symbol,
  cte.current_price,
  cte.day_high,
  cte.day_low,
  vc.volatility
FROM cte
LEFT JOIN volatility_calc vc ON cte.symbol = vc.symbol;


In [0]:
  %sql
WITH cg_hour AS (
  SELECT
      UPPER(symbol)                              AS symbol,
      date_trunc('hour', last_updated)           AS ts_hour,
      current_price,
      market_cap
      -- … other CG fields
  FROM raw.silver_coingecko_data
),

cd_hour AS (
  SELECT
      UPPER(symbol)                              AS symbol,
      to_timestamp(hour_timestamp)               AS ts_hour,
      current_price        AS current_price_cd,
      day_high             AS high_24h,
      day_low              AS low_24h
  FROM silver_coindesk
)

SELECT *
FROM cg_hour cg
JOIN cd_hour cd
  USING (symbol);


In [0]:
df = (
    spark.readStream
    .format("kafka")
    .option("kafka.bootstrap.servers", "localhost:9092")
    .option("subscribe", "test-topic")
    .load()
)

df.printSchema()


root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [0]:
from kafka import KafkaProducer

producer = KafkaProducer(bootstrap_servers='localhost:9092')

for i in range(10):
    msg = f"Message {i}"
    producer.send('test-topic', value=msg.encode('utf-8'))
    print(f"Sent: {msg}")

producer.flush()


---------------------------------------------------------------------------
NoBrokersAvailable                        Traceback (most recent call last)
File <command-7044717791728054>, line 3
      1 from kafka import KafkaProducer
----> 3 producer = KafkaProducer(bootstrap_servers='localhost:9092')
      5 for i in range(10):
      6     msg = f"Message {i}"

File /local_disk0/.ephemeral_nfs/envs/pythonEnv-d26eb304-58c2-4826-bf75-c8d00d6114b3/lib/python3.11/site-packages/kafka/producer/kafka.py:481, in KafkaProducer.__init__(self, **configs)
    478 else:
    479     self._metrics = None
--> 481 client = self.config['kafka_client'](
    482     metrics=self._metrics, metric_group_prefix='producer',
    483     wakeup_timeout_ms=self.config['max_block_ms'],
    484     **self.config)
    486 # Get auto-discovered / normalized version from client
    487 self.config['api_version'] = client.config['api_version']

File /local_disk0/.ephemeral_nfs/envs/pythonEnv-d26eb304-58c2-4826-bf75-c8d

In [0]:
%pip install kafka-python

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
